In [1]:
import pandas as pd
import scimap as sm
import anndata as ad
import tifffile as tiff
import zarr
import dask.array as da

Running SCIMAP  2.3.5


/Users/lukashat/miniforge3/envs/scimap_benchmark/lib/python3.10/site-packages/mpl_scatter_density/__init__.py:4: UserWarning:

pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.

/Users/lukashat/miniforge3/envs/scimap_benchmark/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [8]:
def reorder_columns(df, col_changed, col_position) -> pd.DataFrame:
    """
    Reordering colums. The second input can either be an integer for index or it can be a reference column name. If reference column name is chosen, the column will be placed after the reference column.
    """
    if not isinstance(df, pd.DataFrame):
        raise ValueError("The first argument must be a pandas DataFrame")
    if not isinstance(col_changed, str):
        raise ValueError("The second argument must be a string representing a column name")
    if not isinstance(col_position, (str, int)):
        raise ValueError("The third argument must be either a string representing a column name or an integer representing a column index")
    if isinstance(col_position, int):
        cols = df.columns.tolist()
        if col_changed in cols and col_position <= len(cols) + 1:
            cols.remove(col_changed)
        
            index = col_position
            cols.insert(index, col_changed)
            
            df = df[cols]
    else:
        cols = df.columns.tolist()
        if col_changed in cols and col_position in cols:
            cols.remove(col_changed)
            index = cols.index(col_position)
            cols.insert(index+1, col_changed)
            
            df = df[cols]
    return df

In [2]:
df = pd.read_csv('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/cycif_ovca/quantification/cycif_single_cell_spatial_25.csv', index_col=0)
img_path = '/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/cycif_ovca/img/S012_chemonaive.ome.tif'
#seg_path = 

In [14]:
df

,ID,pERK,FOXP3,LaminB1,pSTAT1,IBA1,MHCII,CD57,CD8a,SMA,...,Ecadherin,FOXOA3,KRAS,X.Position,Y.Position,GlobalCellType2,neighbordood_cluster2,Stage,Patient_code_final,Sample_code_final
1,20284,0.372753,0.337156,0.367110,0.405686,0.408141,0.486107,0.385586,0.366450,0.292237,...,0.356802,0.364615,0.398076,2387,6443,IBA1.CD163.Macrophages,Epithelial_EMT,primary,S013,S013_primary
2,43273,0.401036,0.509924,0.512477,0.480357,0.512132,0.634644,0.512915,0.511404,0.505456,...,0.512238,0.512377,0.511586,3426,7956,Proliferating.epithelial,epithelial_and_proliferating_epithelial,primary,S013,S013_primary
3,562,0.361901,0.503882,0.504551,0.437978,0.505056,0.396402,0.505478,0.504477,0.505909,...,0.504303,0.505878,0.504723,1133,5112,Fibroblast,tumor-stroma-interface,primary,S013,S013_primary
4,29026,0.368904,0.536897,0.545607,0.423388,0.542831,0.629951,0.544818,0.538702,0.528172,...,0.543347,0.544746,0.539554,2746,7921,Proliferating.EMT,Proliferating.epithelial,primary,S013,S013_primary
5,67527,0.388265,0.501220,0.502478,0.415093,0.502006,0.516949,0.502401,0.501875,0.416413,...,0.501650,0.501756,0.500740,4891,2644,Proliferating.EMT,Proliferating.EMT,primary,S013,S013_primary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6422538,36244,0.461307,0.000763,0.001621,0.518292,0.001122,0.228771,0.000895,0.001016,0.000809,...,0.001104,0.001428,0.001950,4097,7476,Epithelial,epithelial_and_proliferating_epithelial,primary,S010,S010_primary
6422539,44801,0.377578,0.000539,0.001472,0.436890,0.000676,0.203623,0.000662,0.000909,0.001035,...,0.001216,0.001379,0.001910,5185,7323,Epithelial,epithelial_and_proliferating_epithelial,primary,S010,S010_primary
6422540,17102,0.485293,0.000816,0.001623,0.530833,0.001099,0.223898,0.001038,0.001117,0.001232,...,0.001024,0.001482,0.002103,2624,7092,Proliferating.epithelial,tumor-stroma-interface,primary,S010,S010_primary
6422541,42468,0.434089,0.000658,0.001377,0.477240,0.001061,0.209598,0.000784,0.001857,0.001966,...,0.000535,0.001446,0.001910,4855,5271,CD8.T.cells,SMA.CD31.positive,primary,S010,S010_primary


In [3]:
with tiff.TiffFile(img_path) as tif:
    zarr_store = tif.aszarr()
    zarr_group = zarr.open(zarr_store, mode='r')
    datasets = zarr_group.attrs['multiscales'][0]['datasets']
    dask_arrays = [
        da.array.from_zarr(zarr_store, component=ds['path'])
        for ds in datasets
    ]
    fully_res_array= dask_arrays[0]

KeyError: 'multiscales'

In [10]:
img = tiff.imread(img_path, aszarr=True, is_ome=False, series = 0, level = 2)

In [12]:
with tiff.TiffFile(img_path) as tif:
    length = len(tif.series)
    print(length)

7


In [12]:
with tiff.TiffFile(img_path) as tif:
    zarr_store = tif.aszarr()
    zarr_group = zarr.open(zarr_store, mode='r')

In [13]:
zarr_group

<zarr.core.Array (52, 15888, 35716) uint16 read-only>

In [ ]:
with open('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/cycif_ova/markers.txt') as f:
    markers = f.read().splitlines()
markers

In [ ]:
df = reorder_columns(df, 'ID', 'Y.Position')
X_columns = df.columns[:df.columns.get_loc('X.Position')]
obs_columns =df.columns[df.columns.get_loc('X.Position'):]
adata = ad.AnnData(
    X=df[X_columns],
    obs=df[obs_columns],
    var=pd.DataFrame(index=X_columns)
)

In [ ]:
sm.pl.image_viewer(image_path=img_path, 
                   adata=adata, 
                   overlay='GlobalCellType2', 
                   point_size=5,
                   y_coordinate='Y.Position',
                   x_coordinate='X.Position',
                   subset='S012_primary',
                   imageid='Sample_code_final',
                   point_color='white')